In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plot
import seaborn as sns
import sklearn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Importing data

In [ ]:
# Importing data
test_data = pd.read_csv("/kaggle/input/ace-class-assignment/Test.csv", header=0, sep=",")
amp_train = pd.read_csv("/kaggle/input/ace-class-assignment/AMP_TrainSet.csv", header=0, sep=",")
print(amp_train)
#print(test)




In [ ]:
amp_train.columns

In [ ]:
#test_amp =amp_train[['FULL_Charge', 'FULL_AcidicMolPerc','FULL_OOBM850104','FULL_DAYM780201', 'AS_MeanAmphiMoment','CLASS']]
#print(test_amp)
#test_data=test_data[['FULL_Charge', 'FULL_AcidicMolPerc','FULL_OOBM850104','FULL_DAYM780201', 'AS_MeanAmphiMoment']]
#X=test_amp.drop("CLASS",axis=1)
#Y=test_amp.CLASS
#model=GaussianNB()   
#model.fit(X,Y)
#kj_results=model.predict(test_data)

#print(test_data2)
#kj_results

## General data attributes and descriptive statistics

In [ ]:
#Describing general data attributes
train_dim=amp_train.shape #gets the dimensions of the tranining dataset
test_dim= test_data.shape #gets the dimensions of the tesing dataset
print(train_dim)
print(test_dim)

atr_types=amp_train.dtypes #gets the data type for each column in training dataset
print(atr_types)
amp_train.isnull().sum() # gets the sum of the missing values/observations in each column


In [ ]:
# Descriptive statistics 
stats=amp_train.describe()
print(stats)

# Class distributions

Sometimes, observations from the different classes may not be equal, these may need special handling, so its important to note the class distributions, as if they are uneven, they may bias our model

In [ ]:
# CLass distibutions 
cdists= amp_train.groupby('CLASS').size() # groups the observation by the column of class, and counts all the observations including null values
print(cdists)

#plots to visualize the class distribution
amp_train.groupby('CLASS').size().plot(kind="pie")

The observations from each class seem to be equal

# Correlation between the different attributes
Basically, we need to examine our attributes for correlation as highly correlated attributes present the same data to the algorithm. Removing one may speed learning of the algorithm (less dimensions more speed)

In [ ]:
# Correlation between attributes 
correlations = amp_train.corr(method="pearson") # calculates pairwaise correlation for the attributes 
print(correlations)



FULL_AURR980107 and FULL_AcidicMolPerc have a correlation coefficent of 0.79, this might mean these two attributes may be some what correlated. Same goes for AS_DAYM780201 and FULL_DAYM780201 (correlation coefficient is 0.894191)

In [ ]:
# Plots to visualize the correlations
from matplotlib import rcParams
sns.heatmap(correlations, annot=True)
rcParams['figure.figsize'] = 11.7,8.27


# Data skewness

Most machine learning algorithms assume that data from the attributes is normally distributed, so identifying attributes with data that has a left or right skew, can be important as this can be easily corrected

In [ ]:
sk = amp_train.skew() # calculates the skew for each attribute
print(sk)

Negative values like those of FULL_DAYM780201, FULL_OOBM850104,AS_DAYM780201, AS_FUKS010112, indicate that these attributes have values skewed more to the left
Values closer to zero are indicative of a less or no skew, while positive values would mean that that particular attribute is skewed more to the right

 # Visualising data distributions with plots

In [ ]:
# Plot one, histogram to show the distributions
import matplotlib as plot 
from matplotlib import pyplot
amp_train.hist(layout=(4,3), figsize=(16,16))
plot.pyplot.subplots_adjust(wspace=0.4, hspace=0.5)
pyplot.show()

In [ ]:
# Plot two density plots can show the distributions better
amp_train.plot(kind="density",subplots=True,layout=(4,3),sharex=False,sharey=False, figsize=(16,16))
#amp_train.plot(kind="density",subplots=False,layout=(4,3),sharex=True,sharey=True)
pyplot.show()


# Data transformations
Using square roots to transform the attributes with negative values into postives

In [ ]:

neg1=amp_train.iloc[:,0]
print(neg1)
tneg1=neg1**2
tneg1.plot(kind="density")
amp_train.iloc[:,0]=tneg1



neg2=amp_train.iloc[:,5]
print(neg2)
tneg2=neg2**2
tneg2.plot(kind="density")
amp_train.iloc[:,5]=tneg2

## test
neg3=test_data.iloc[:,0]
print(neg3)
tneg3=neg3**2
tneg3.plot(kind="density")
test_data.iloc[:,0]=tneg3



neg4=test_data.iloc[:,5]
print(neg4)
tneg4=neg4**2
tneg4.plot(kind="density")
test_data.iloc[:,5]=tneg4

distributions for AS_MeanAmphiMoment,FULL_AcidicMolPerc and NT_EFC195 dont look so good

# Feature selection




In [ ]:
 #test one selecting out features with low variance
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
k=sel.fit_transform(amp_train)
names=amp_train.columns[sel.get_support(indices=True)]
print(k.shape)
amp_train2=pd.DataFrame(k,columns=names)
print(amp_train2)

In [ ]:
#test two
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test_array = amp_train2.values

test_atr = test_array[:,0:7]
class_atr = test_array[:,7]


test = SelectKBest(score_func=chi2, k=5)
fit = test.fit(test_atr, class_atr)


print(fit.scores_)
features = fit.transform(test_atr)
names2=amp_train2.columns[fit.get_support(indices=True)]
amp_train3=pd.DataFrame(features,columns=names2)
print(amp_train3[5:])





In [ ]:
# Test three Recursive feature elimination

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier

model1 = LogisticRegression()
model2=RandomForestClassifier()

rfe = RFE(model1, 4)
fit2 = rfe.fit(test_atr, class_atr)

rfe2= RFE(model2, 4)
fit3 = rfe2.fit(test_atr, class_atr)




names3=amp_train2.columns[fit2.get_support(indices=True)]

names4=amp_train2.columns[fit3.get_support(indices=True)]

print(names3)
print(names4)
print(names2)
print(amp_train.columns)
print(amp_train2)





In [ ]:
test_amp =amp_train.loc[:,['FULL_Charge', 'FULL_AcidicMolPerc',"FULL_OOBM850104",'FULL_DAYM780201', 'AS_MeanAmphiMoment','CLASS']]
print(test_amp)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


## Logistic regression using cross validation

In [ ]:

X=test_amp.iloc[:,0:5]
Y=test_amp.iloc[:,5]
folds=10
kfold = KFold(n_splits=folds, random_state=7, shuffle=True,)
model = LogisticRegression()
results = cross_val_score(model, X, Y, cv=kfold)

print(("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0))





In [ ]:
test_data = pd.read_csv("/kaggle/input/ace-class-assignment/Test.csv", header=0, sep=",")
amp_train = pd.read_csv("/kaggle/input/ace-class-assignment/AMP_TrainSet.csv", header=0, sep=",")
test_amp =amp_train.loc[:,['FULL_Charge', 'FULL_AcidicMolPerc',"FULL_OOBM850104",'FULL_DAYM780201', 'AS_MeanAmphiMoment','CLASS']]

# Logistic regression with a test train split

In [ ]:
from sklearn.metrics import matthews_corrcoef
test_amp=test_amp.values
#X=test_amp.iloc[:,0:5]
X=test_amp[:,0:5]
Y=test_amp[:,5]

#Y=test_amp.iloc[:,5]
#print(Y)
test_size = 0.4
seed = 25
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,random_state=seed, shuffle=True)
model = LogisticRegression()
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result = model.score(X_test, Y_test)
print("Accuracy: ",  (result*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model.predict(X_test),Y_test))
#print(test_data)

test_data2=test_data.loc[:,['FULL_Charge', 'FULL_AcidicMolPerc','FULL_DAYM780201', 'FULL_OOBM850104', 'AS_MeanAmphiMoment']]
test_data2=test_data2.values    
#model1 = LogisticRegression()    
#model1.fit(X,Y)
kj_results=model.predict(test_data2)

#print(test_data2)
kj_results

In [ ]:
df = pd.DataFrame(kj_results)
df.columns = ['CLASS']
df.index.name = 'Index'
df['CLASS']=df['CLASS'].map({0.0:False,1.0:True})
print(df)
#X=test_amp.iloc[:,0:5]
#=test_amp.iloc[:,5]
df.to_csv("kj.csv")
df["CLASS"].unique()
#print(df.groupby("CLASS").size()[0].sum())
#print(df.groupby("CLASS").size()[1].sum())
df['CLASS'].value_counts()

# Gausian/Naive Bayes

In [ ]:
model=GaussianNB()
from sklearn.metrics import matthews_corrcoef
test_size = 0.4
seed = 25
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,
random_state=seed)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
result = model.score(X_test, Y_test)
print("Accuracy: ",  (result*100.0))
print(matrix)
print('MCC: ', matthews_corrcoef(model.predict(X_test),Y_test))
#print(test_data)

X=test_amp.iloc[:,0:5]
Y=test_amp.iloc[:,5]
model=GaussianNB()   
model.fit(X,Y)
kj_results=model.predict(test_data2)

#print(test_data2)
kj_results

# Linear Discriminant Analysis

In [ ]:
model=LinearDiscriminantAnalysis()
folds=10
kfold = KFold(n_splits=folds, random_state=7, shuffle=True,)
results = cross_val_score(model, X, Y, cv=kfold)
print(("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0))


# Support Vector Machines

In [ ]:
model=SVC()
folds=10
kfold = KFold(n_splits=folds, random_state=7, shuffle=True,)
results = cross_val_score(model, X, Y, cv=kfold)
print(("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0))